# Claude Skills 入门指南

学习如何使用Claude的Skills功能来创建专业文档、分析数据，以及通过Excel、PowerPoint和PDF生成功能自动化业务流程。

> **查看实际应用：** 你将在本文中学到的Skills为Claude的文件创建功能提供了强大支持！查看 **[Claude创建文件](https://www.anthropic.com/news/create-files)** 了解这些技能如何使Claude能够在Claude.ai中直接创建和编辑文档。

## 目录

1. [环境设置与安装](#setup)
2. [理解Skills](#understanding)
3. [探索可用技能](#discovering)
4. [快速入门：Excel](#excel-quickstart)
5. [快速入门：PowerPoint](#powerpoint-quickstart)
6. [快速入门：PDF](#pdf-quickstart)
7. [故障排除](#troubleshooting)

## 1. 环境设置与安装 {#setup}

### 先决条件

在开始之前，请确保您拥有：
- Python 3.8或更高版本
- 来自[console.anthropic.com](https://console.anthropic.com/)的Anthropic API密钥

### 环境设置（仅首次需要）

**如果您尚未设置环境**，请按以下步骤操作：

#### 步骤1：创建虚拟环境

```bash
# 导航到skills目录
cd /path/to/claude-cookbooks/skills

# 创建虚拟环境
python -m venv venv

# 激活它
source venv/bin/activate  # 在macOS/Linux上
# 或者
venv\Scripts\activate     # 在Windows上
```

#### 步骤2：安装依赖

```bash
# 激活venv后，安装requirements文件
pip install -r requirements.txt
```

#### 步骤3：在VSCode/Jupyter中选择内核

**在VSCode中：**
1. 打开此notebook
2. 点击右上角的内核选择器（例如"Python 3.11.x"）
3. 选择"Python环境..."
4. 选择`./venv/bin/python`解释器

**在Jupyter中：**
1. 从内核菜单 → 更换内核
2. 选择与您的venv匹配的内核

#### 步骤4：配置API密钥

```bash
# 复制示例文件
cp .env.example .env

# 编辑.env并添加您的API密钥：
# ANTHROPIC_API_KEY=sk-ant-api03-...
```

### 快速安装检查

运行下面的单元格来验证您的环境设置是否正确：

**如果您在上面看到任何❌或⚠️警告**，请在继续之前完成设置步骤。

**如果anthropic SDK版本太旧（需要0.71.0或更高版本）：**
```bash
pip install anthropic>=0.71.0
```
然后**重新启动Jupyter内核**以使用新版本。

---

### API配置

现在让我们加载API密钥并配置客户端：

### API配置

**⚠️重要**：在skills目录中创建`.env`文件：

```bash
# 复制示例文件
cp ../.env.example ../.env
```

然后编辑`../.env`以添加您的Anthropic API密钥。

In [ ]:
import os
import sys
from pathlib import Path

# 添加父目录到导入路径
sys.path.insert(0, str(Path.cwd().parent))


from anthropic import Anthropic
from dotenv import load_dotenv

# 导入我们的文件工具
from file_utils import (
    download_all_files,
    extract_file_ids,
    get_file_info,
    print_download_summary,
)

# 从父目录加载环境变量
load_dotenv(Path.cwd().parent / ".env")

API_KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-5")

if not API_KEY:
    raise ValueError(
        "未找到ANTHROPIC_API_KEY。请将../.env.example复制到../.env并添加您的API密钥。"
    )

# 初始化客户端
# 注意：当使用Skills时，我们将按请求添加beta头部
client = Anthropic(api_key=API_KEY)

# 如果不存在则创建outputs目录
OUTPUT_DIR = Path.cwd().parent / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ API密钥已加载")
print(f"✓ 使用模型：{MODEL}")
print(f"✓ 输出目录：{OUTPUT_DIR}")
print("\n📝 注意：使用Skills时将按请求添加beta头部")

### 测试连接

让我们验证我们的API连接是否工作：

In [ ]:
# 简单测试验证API连接
test_response = client.messages.create(
    model=MODEL,
    max_tokens=100,
    messages=[
        {
            "role": "user",
            "content": "如果您能读取此信息，请回复'连接成功！'。",
        }
    ],
)

print("API测试响应：")
print(test_response.content[0].text)
print(
    f"\n✓ 令牌使用情况：{test_response.usage.input_tokens} 输入，{test_response.usage.output_tokens} 输出"
)

## 2. 理解Skills {#understanding}

### 什么是Skills？

**Skills**是组织化的指令包、可执行代码和资源，为Claude提供针对特定任务的专业能力。可以将它们视为Claude可以动态发现和加载的"专业技能包"。

📖 阅读我们的工程博客文章[用Skills装备真实世界的智能体](https://www.anthropic.com/engineering/equipping-agents-for-the-real-world-with-agent-skills)

### 为什么Skills很重要

在学习MCPs（模型上下文协议）和工具后，您可能想知道为什么Skills很重要：

- **Skills比单个工具更高级** - 它们结合了指令、代码和资源
- **Skills是可组合的** - 多个技能可以无缝协同工作
- **Skills很高效** - 渐进式披露意味着您只需为您使用的内容付费
- **Skills包含经过验证的代码** - 帮助脚本可靠运行，节省时间并减少错误

### 主要优势

- **专业级性能**：无需学习曲线即可提供专业结果
- **经过验证的帮助脚本**：Skills包含经过测试、可工作的代码，Claude可以立即使用
- **组织知识**：打包公司工作流程和最佳实践
- **成本效益**：渐进式披露最小化令牌使用
- **可靠性**：预测试脚本意味着更少的错误和一致的结果
- **节省时间**：Claude使用现有解决方案而不是从头生成代码
- **可组合性**：多个技能协同工作以实现复杂工作流

### 渐进式披露架构

Skills使用三级加载模型：

![渐进式披露 - Skills如何加载](../assets/prog-disc-1.png)

1. **元数据**（名称：64字符，描述：1024字符）：Claude看到技能名称和描述
2. **完整指令**（<5k令牌）：当技能相关时加载
3. **链接文件**：仅在需要时加载的额外资源

![渐进式披露阶段](../assets/prog-disc-2.png)

这在按需提供深度专业知识的同时保持操作高效。最初，Claude仅看到SKILL.md的YAML前导部分的元数据。只有当技能相关时，Claude才会加载完整内容，包括任何帮助脚本和资源。

### 技能类型

| 类型 | 描述 | 示例 |
|------|-------------|----------|
| **Anthropic管理** | 由Anthropic维护的预构建技能 | `xlsx`、`pptx`、`pdf`、`docx` |
| **自定义** | 针对特定工作流的用户定义技能 | 品牌指南、财务模型 |

### Skills概念概览

![Skills概念图](../assets/skills-conceptual-diagram.png)

此图说明了：
- **技能目录结构**：Skills如何与SKILL.md和支持文件组织
- **YAML前导**：Claude最初看到的元数据
- **渐进式加载**：如何按需发现和加载Skills
- **可组合性**：多个Skills在单个请求中协同工作

### Skills如何与代码执行一起工作

Skills需要启用**代码执行**工具。以下是典型的工作流：

```python
# 使用client.beta.messages.create()以支持Skills
response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=4096,
    container={
        "skills": [
            {"type": "anthropic", "skill_id": "xlsx", "version": "latest"}
        ]
    },
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    messages=[{"role": "user", "content": "创建Excel文件..."}],
    # 使用betas参数而不是extra_headers
    betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"]
)
```

**发生了什么：**
1. Claude接收带有加载的xlsx技能的请求
2. Claude使用代码执行创建文件
3. 响应包含创建文件的`file_id`
4. 您使用**文件API**下载文件

**重要：Beta API**
- 使用`client.beta.messages.create()`（不是`client.messages.create()`）
- `container`参数仅在beta API中可用
- 使用`betas`参数启用beta功能：
  - `code-execution-2025-08-25` - 启用代码执行
  - `files-api-2025-04-14` - 下载文件必需
  - `skills-2025-10-02` - 启用Skills功能

⚠️**注意**：使用Skills时，您必须在请求中包含code_execution工具。

### 令牌使用优化

与在提示中提供指令相比，Skills大幅减少令牌使用：

| 方法 | 令牌成本 | 性能 |
|----------|------------|-------------|
| 手动指令 | 5,000-10,000令牌/请求 | 可变质量 |
| Skills（仅元数据） | 最小（仅名称/描述） | 专业级 |
| Skills（完整加载） | 当使用技能时约5,000令牌 | 专业级 |

**大获全胜：** 您可以将多个技能打包到提示中，而不会使其臃肿。每个技能仅在您实际使用它之前花费元数据（名称+描述）。

**示例**：创建带有格式的Excel文件
- 没有Skills：约8,000令牌来提前解释所有Excel功能
- Skills：最初最小元数据开销，仅在调用Excel技能时约5,000令牌
- **关键洞察**：98%的节省适用于初始上下文。一旦使用技能，就会加载完整指令。

**额外优势：**
- Skills包含已知可工作的帮助脚本，提高可靠性
- Claude通过使用经过验证的代码模式而不是从头生成来节省时间
- 您获得更一致、专业的结果

### ⏱️ 预期生成时间

**⚠️重要**：使用Skills的文档生成需要代码执行和文件创建，这需要时间。请耐心等待，让单元格完成。

**观察到的生成时间：**
- **Excel文件**：约2分钟（带有图表和格式）
- **PowerPoint演示**：约1-2分钟（简单的2页演示带图表）
- **PDF文档**：约40-60秒（简单文档）

**预期情况：**
- 单元格运行时将显示`[*]`
- 您可能会看到"执行中..."状态1-2分钟
- **不要中断单元格** - 让它完全完成

**💡建议：**
1. **从简单开始**：从最小示例开始验证您的设置
2. **逐步增加复杂性**：增量添加功能
3. **保持耐心**：操作通常需要40秒到2分钟
4. **注意**：非常复杂的文档可能需要更长时间 - 保持示例集中

## 3. 探索可用技能 {#discovering}

### 列出所有内置技能

让我们发现有哪些Anthropic管理的技能可用：

In [ ]:
# 列出所有可用的Anthropic技能
# 注意：技能API需要skills beta头部
client_with_skills_beta = Anthropic(
    api_key=API_KEY, default_headers={"anthropic-beta": "skills-2025-10-02"}
)

skills_response = client_with_skills_beta.beta.skills.list(source="anthropic")

print("可用的Anthropic管理技能：")
print("=" * 80)

for skill in skills_response.data:
    print(f"\n📦 技能ID：{skill.id}")
    print(f"   标题：{skill.display_title}")
    print(f"   最新版本：{skill.latest_version}")
    print(f"   创建时间：{skill.created_at}")

    # 获取版本详细信息
    try:
        version_info = client_with_skills_beta.beta.skills.versions.retrieve(
            skill_id=skill.id, version=skill.latest_version
        )
        print(f"   名称：{version_info.name}")
        print(f"   描述：{version_info.description}")
    except Exception as e:
        print(f"   （无法获取版本详细信息：{e}）")

print(f"\n\n✓ 找到{len(skills_response.data)}个Anthropic管理技能")

### 理解技能元数据

每个技能都有：
- **skill_id**：唯一标识符（例如，"xlsx"、"pptx"）
- **version**：版本号或"latest"
- **name**：人类可读的名称
- **description**：技能的功能描述
- **directory**：技能的文件夹结构

### 版本控制策略

- 对Anthropic技能使用"latest"（推荐）
- Anthropic自动更新技能
- 为生产稳定性固定特定版本
- 自定义技能使用时间戳作为版本

### 示例：月度预算电子表格

我们将从一个简单的示例开始 - 一个简单的一行代码和一个详细请求。

#### 简单示例（1-2行）
首先，让我们看看Skills如何通过最小提示工作：

```python
# 简单提示 - Skills处理复杂性
prompt = "创建包含收入数据和图表的季度销售报告Excel文件"
```

#### 详细示例
要获得更多控制，您可以提供特定要求：
- 收入和支出类别
- 总计公式
- 基本格式

### 示例：月度预算电子表格

我们将创建一个简单的预算电子表格，包含：
- 收入和支出类别
- 总计公式
- 基本格式

**⏱️注意**：Excel生成通常需要**1-2分钟**（带有图表和格式）。单元格运行时将显示`[*]` - 请耐心等待！

In [ ]:
# 创建Excel预算电子表格
excel_response = client.beta.messages.create(  # 注意：使用beta.messages以支持Skills
    model=MODEL,
    max_tokens=4096,
    container={"skills": [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}]},
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    messages=[
        {
            "role": "user",
            "content": """创建一个包含以下内容的月度预算Excel电子表格：

收入：
- 工资：$5,000
- 自由职业：$1,200
- 投资：$300

支出：
- 租金：$1,500
- 水电费：$200
- 食品杂货：$600
- 交通费：$300
- 娱乐：$400
- 储蓄：$1,000

包括：
1. 计算总收入和总总支出的公式
2. 净储蓄公式（收入-支出）
3. 正确格式化货币值
4. 添加显示收入与支出的简单柱状图
5. 使用带有标题的专业格式
""",
        }
    ],
    # 使用betas参数用于beta功能
    betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"],
)

print("Excel响应：")
print("=" * 80)
for content in excel_response.content:
    if content.type == "text":
        print(content.text)
    elif content.type == "tool_use":
        print(f"\n🔧 工具：{content.name}")
        if hasattr(content, "input"):
            print(f"   输入预览：{str(content.input)[:200]}...")

print("\n\n📊 令牌使用情况：")
print(f"   输入：{excel_response.usage.input_tokens}")
print(f"   输出：{excel_response.usage.output_tokens}")

### 下载Excel文件

现在让我们提取file_id并下载生成的Excel文件：

In [ ]:
# 从响应中提取文件ID
file_ids = extract_file_ids(excel_response)

if file_ids:
    print(f"✓ 找到{len(file_ids)}个文件\n")

    # 下载所有文件
    results = download_all_files(
        client, excel_response, output_dir=str(OUTPUT_DIR), prefix="budget_"
    )

    # 打印摘要
    print_download_summary(results)

    # 显示文件详细信息
    for file_id in file_ids:
        info = get_file_info(client, file_id)
        if info:
            print("\n📄 文件详细信息：")
            print(f"   文件名：{info['filename']}")
            print(f"   大小：{info['size'] / 1024:.1f} KB")
            print(f"   创建时间：{info['created_at']}")
else:
    print("❌ 响应中未找到文件")
    print("\n调试：响应内容类型：")
    for i, content in enumerate(excel_response.content):
        print(f"  {i}. {content.type}")

**✨ 刚刚发生了什么？**

1. Claude使用`xlsx`技能创建了专业的Excel文件
2. 技能处理了所有Excel特定的格式和公式
3. 文件在Claude的代码执行环境中创建
4. 我们从响应中提取了`file_id`
5. 我们使用文件API下载了文件
6. 文件现在保存在`outputs/budget_*.xlsx`

打开Excel文件查看结果！

## 5. 快速入门：PowerPoint {#powerpoint-quickstart}

现在让我们使用`pptx`技能创建PowerPoint演示。

### 示例：收入演示

#### 简单示例（1行）
```python
# 最小提示 - 让Skills处理细节
prompt = "创建包含3页关于第三季度结果的执行摘要演示"
```

#### 详细示例
**注意**：这故意保持简单（2页，1个图表）以最小化生成时间并演示核心功能。

### 示例：简单收入演示

**注意**：这故意保持简单（2页，1个图表）以最小化生成时间并演示核心功能。

In [ ]:
# 创建PowerPoint演示
pptx_response = client.beta.messages.create(
    model=MODEL,
    max_tokens=4096,
    container={"skills": [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}]},
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    messages=[
        {
            "role": "user",
            "content": """创建一个简单的2页PowerPoint演示：

第1页：标题页
- 标题："2025年第三季度结果"
- 副标题："Acme公司"

第2页：收入概览
- 标题："季度收入"
- 添加一个简单的柱状图显示：
  - Q1：$12M
  - Q2：$13M
  - Q3：$14M

使用简洁、专业的格式。
""",
        }
    ],
    betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"],
)

print("PowerPoint响应：")
print("=" * 80)
for content in pptx_response.content:
    if content.type == "text":
        print(content.text)

print("\n\n📊 令牌使用情况：")
print(f"   输入：{pptx_response.usage.input_tokens}")
print(f"   输出：{pptx_response.usage.output_tokens}")

### 下载PowerPoint文件

In [ ]:
# 下载PowerPoint文件
file_ids = extract_file_ids(pptx_response)

if file_ids:
    results = download_all_files(
        client, pptx_response, output_dir=str(OUTPUT_DIR), prefix="q3_review_"
    )

    print_download_summary(results)

    print("\n✅ 在PowerPoint或Google幻灯片中打开演示查看！")
else:
    print("❌ 响应中未找到文件")

**⏱️注意**：PDF生成通常需要**1-2分钟**（简单文档）。单元格运行时将显示`[*]` - 请耐心等待！

### 示例：PDF文档

#### 简单示例（1行）
```python
# 快速PDF生成
prompt = "为$500咨询服务创建专业发票PDF"
```

#### 详细示例：收据
**注意**：这故意保持简单以确保清洁格式。

### 示例：简单收据

**注意**：这故意保持简单以确保清洁格式。

In [ ]:
# 创建PDF收据
pdf_response = client.beta.messages.create(
    model=MODEL,
    max_tokens=4096,
    container={"skills": [{"type": "anthropic", "skill_id": "pdf", "version": "latest"}]},
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    messages=[
        {
            "role": "user",
            "content": """创建一个简单的收据PDF：

收据

Acme公司
日期：2025年1月15日
收据号：RCT-2025-001

客户：Jane Smith

商品：
- 产品A：$50.00
- 产品B：$75.00
- 产品C：$25.00

小计：$150.00
税费（8%）：$12.00
总计：$162.00

谢谢您的业务！

使用简洁、清洁的格式，有清晰的区域划分。
""",
        }
    ],
    betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"],
)

print("PDF响应：")
print("=" * 80)
for content in pdf_response.content:
    if content.type == "text":
        print(content.text)

print("\n\n📊 令牌使用情况：")
print(f"   输入：{pdf_response.usage.input_tokens}")
print(f"   输出：{pdf_response.usage.output_tokens}")

### 下载并验证PDF

In [ ]:
# 下载PDF文件
file_ids = extract_file_ids(pdf_response)

if file_ids:
    results = download_all_files(
        client, pdf_response, output_dir=str(OUTPUT_DIR), prefix="receipt_"
    )

    print_download_summary(results)

    # 验证PDF完整性
    for result in results:
        if result["success"]:
            file_path = result["output_path"]
            file_size = result["size"]

            # 基本PDF验证
            with open(file_path, "rb") as f:
                header = f.read(5)
                if header == b"%PDF-":
                    print(f"\n✅ PDF文件有效：{file_path}")
                    print(f"   文件大小：{file_size / 1024:.1f} KB")
                else:
                    print(f"\n⚠️ 文件可能不是有效的PDF：{file_path}")
else:
    print("❌ 响应中未找到文件")

## 7. 故障排除 {#troubleshooting}

### 常见问题及解决方案

### 问题1：API密钥未找到

**错误：**
```
ValueError: 未找到ANTHROPIC_API_KEY
```

**解决方案：**
1. 确保在父目录中存在`.env`文件
2. 检查`ANTHROPIC_API_KEY=sk-ant-api03-...`是否设置
3. 创建/编辑`.env`后重启Jupyter内核

### 问题2：不识别container参数

**错误：**
```
TypeError: Messages.create()得到了意外的关键字参数'container'
```

**解决方案：**
使用`client.beta.messages.create()`而不是`client.messages.create()`：
```python
# ✅ 正确 - 使用beta.messages
response = client.beta.messages.create(
    model=MODEL,
    container={"skills": [...]},
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    messages=[...],
    betas=["code-execution-2025-08-25", "files-api-2025-04-14", "skills-2025-10-02"]
)

# ❌ 不正确 - 常规消息不支持container
response = client.messages.create(
    model=MODEL,
    container={"skills": [...]},  # 错误！
    messages=[...]
)
```

### 问题3：Skills Beta需要代码执行工具

**错误：**
```
BadRequestError: Skills beta需要在请求中包含code_execution工具。
```

**解决方案：**
使用Skills时，您必须包含code_execution工具：
```python
# ✅ 正确
response = client.beta.messages.create(
    model=MODEL,
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    messages=[...],
    betas=["...", "skills-2025-10-02"]
)

# ❌ 不正确 - 缺少code_execution工具
response = client.beta.messages.create(
    model=MODEL,
    messages=[...],
    betas=["...", "skills-2025-10-02"]
)
```

### 问题4：响应中未找到文件

**错误：**
```
❌ 响应中未找到文件
```

**解决方案：**
1. 检查请求中是否包含代码执行工具
2. 验证技能是否已加载（检查响应内容）
3. 确保任务实际需要文件创建
4. 在响应文本中查找错误消息

### 问题5：文件下载失败

**错误：**
```
Error retrieving file: File not found
```

**解决方案：**
1. 文件在Anthropic服务器上的生存期可能有限
2. 创建后立即下载文件
3. 检查file_id是否从响应中正确提取
4. 验证Files API beta是否包含在betas列表中

### 令牌优化提示

1. **对Anthropic技能使用"latest"版本** - 自动优化
2. **批处理操作** - 尽可能在一次对话中创建多个文件
3. **重用容器** - 使用来自先前响应的`container.id`以避免重新加载技能
4. **保持具体** - 清晰的指令意味着更少的迭代

### API速率限制

如果遇到速率限制：
- 对重试实施指数退避
- 对多个文件使用批处理
- 考虑升级API层级以获得更高限制

## 下一步

🎉 **恭喜！** 您已经学会了Claude Skills的基础知识。

### 查看技能实际应用

查看官方公告，了解这些技能如何为Claude的文件创建功能提供支持：
- **[Claude创建文件](https://www.anthropic.com/news/create-files)** - 了解技能如何使Claude能够直接在Excel、PowerPoint和PDF文件中创建和编辑

### 继续学习：

- **[Notebook 2：金融应用](02_skills_financial_applications.ipynb)** - 使用财务数据的实际业务用例
- **[Notebook 3：自定义技能开发](03_skills_custom_development.ipynb)** - 构建您自己的专业技能

### 支持文章：

- 📚 **[使用Skills教Claude您的工作方式](https://support.claude.com/en/articles/12580051-teach-claude-your-way-of-working-using-skills)** - 使用Skills的用户指南
- 🛠️ **[如何通过对话创建技能](https://support.claude.com/en/articles/12599426-how-to-create-a-skill-with-claude-through-conversation)** - 交互式技能创建指南

### 资源：

- [Claude API文档](https://docs.anthropic.com/en/api/messages)
- [Skills文档](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/overview)
- [Skills最佳实践](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/best-practices)
- [文件API文档](https://docs.claude.com/en/api/files-content)
- [Claude支持](https://support.claude.com)

### 尝试这些实验：

1. 从简单的一行提示开始查看技能实际效果
2. 修改预算示例以包含更多类别
3. 使用您自己的数据创建演示
4. 生成结合文本和表格的PDF报告
5. 在单个请求中同时使用多个技能